# RNN Model Development

In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Masking, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from datetime import datetime, timedelta


# GBQ importing data

In [319]:
list_of_dfs = ["ban", "cummies", "dinu", "doge",
"doggy", "elon", "erc20", "ftm", "grlc", "hoge",
"lowb", "mona", "samo", "shib", "shibx", "smi",
"wow", "yooshi","yummy"]

project_id = 'crypto-prediction-333213'

def get_data():
    
    dfs = []

    for df in list_of_dfs:

        sql = f"""
        SELECT *
        FROM `crypto-prediction-333213.crypto_BQDB.{df}`;
        """

        dfs.append(pd.read_gbq(sql, project_id=project_id, dialect='standard'))
        
    return dfs

def data_cleaning(dfs):
    
    # takes as input the list of dataframes

    dfs_new = []

    for i in range(0,len(dfs)):
        df = dfs[i]
        price_col = [i for i in df.columns if "price" in i][0]
        
        # data cleaning
        df = df.sort_values(by="datetime")
        df = df.drop_duplicates(keep= "last")
        df = df.set_index("datetime")
        df[price_col].interpolate(method="linear", inplace= True)
        
        # adding financial indicators
        df["hourly_pct_change"] = df[price_col].pct_change(1)
        df["MA12_hours"] = df[price_col].rolling(window=12).mean()
        df["MA72_hours"] = df[price_col].rolling(window=72).mean()
        df["MAshort_over_long"] = df["MA12_hours"] > df["MA72_hours"]
        df["MAshort_over_long"] = df["MAshort_over_long"].apply(lambda x: 1 if x == True else 0)
        
        cols = [i for i in df.columns[2:]]
        for col in cols:
            df[col] = df[col].fillna(-99)
        
        dfs_new.append(df)
    
    return dfs_new



In [320]:
dfs = get_data()
dfs = data_cleaning(dfs)

In [321]:
dfs[0]

,price_ban,tweets_ban,hourly_pct_change,MA12_hours,MA72_hours,MAshort_over_long
datetime,,,,,,
2021-09-09 01:00:00+00:00,0.008674,1,-99.000000,-99.000000,-99.000000,0
2021-09-09 02:00:00+00:00,0.008572,4,-0.011752,-99.000000,-99.000000,0
2021-09-09 03:00:00+00:00,0.008527,2,-0.005172,-99.000000,-99.000000,0
2021-09-09 04:00:00+00:00,0.008642,2,0.013462,-99.000000,-99.000000,0
2021-09-09 05:00:00+00:00,0.008681,0,0.004483,-99.000000,-99.000000,0
...,...,...,...,...,...,...
2021-12-01 12:00:00+00:00,0.017118,2,0.012066,0.016921,0.016797,1
2021-12-01 13:00:00+00:00,0.017245,2,0.007433,0.016944,0.016824,1
2021-12-01 14:00:00+00:00,0.017245,1,0.000030,0.016976,0.016851,1


# Reshaping Data

In [322]:
horizon = 24

history_size = 48

def get_X_y(history_size, horizon, dataset):

    X = []
    y = []

    for i in range(0, dataset.shape[0]-history_size-horizon):
        X.append(dataset[i:i+history_size,:])
        y.append(dataset[i+history_size: i+history_size+horizon,0]) # should be series of next 24 values for 24h forecast

    return np.array(X), np.array(y)

def reshape_data(dfs, history_size):

    # scaling the data & get_X_y for all datasets
    X = []
    y = []
    scalers = []

    for df in dfs:

        # log transform
        price_col = [i for i in df.columns if "price" in i][0]
        df[price_col] = np.log(df[price_col])
        
        data = df.values
        
        # scaling
        scaler = MinMaxScaler()
        
        scaler.fit(data)
        data = scaler.transform(data)
        
        # get X and y
        X_arr, y_arr = get_X_y(history_size, horizon, data)
        
        X.append(X_arr)
        y.append(y_arr)
        scalers.append(scaler)

    # stacking data three-dimensionally
    X = np.stack(X, axis= 2).reshape(len(X[0]),history_size,-1)
    y = np.stack(y, axis = 2).reshape(len(y[0]),-1)

    return X, y, scalers


In [323]:
dfs_tmp = dfs.copy()

In [324]:
X, y, scalers = reshape_data(dfs_tmp, history_size) 

In [325]:
X.shape

(1936, 48, 114)

In [326]:
y.shape

(1936, 456)

In [327]:
import joblib

joblib.dump(scalers, open('scaler.joblib', 'wb'))
print("scaler saved")

scaler saved


# Modeling

In [328]:
def modeling(coins, horizon):    
    # instantiating a model
    model = Sequential()

    # first network layer
    model.add(Masking(mask_value=-99))
    model.add(GRU(units = 100, activation= "tanh", return_sequences= True))
    model.add(Dropout(0.2))

    # network layer's 2 - 5
    model.add(GRU(units= 50, activation= "tanh", return_sequences= True))
    model.add(Dropout(0.2))
    #model.add(LSTM(units= 10, activation= "tanh", return_sequences= True))
    #model.add(Dropout(0.2))
    model.add(LSTM(units= 20, activation= "tanh", return_sequences= False))
    model.add(Dropout(0.2))

    # network output layer
    # predicting future_horizon = 24 hours
    # for two cryptos
    model.add(Dense(coins*horizon, activation= "linear")) 

    model.compile(optimizer= "adam", loss= "mse")

    return model

In [329]:
model = modeling(19,24)

In [330]:
model

In [332]:
es = EarlyStopping(patience = 50, restore_best_weights= True)

history = model.fit(X, 
          y,
          validation_split= 0.2,
          epochs = 1000,
          batch_size= 64,
          callbacks= [es],
          verbose= 1)

Epoch 1/1000
25/25 [==============================] - 1s 49ms/step - loss: 0.1013 - val_loss: 0.0650
Epoch 2/1000
25/25 [==============================] - 1s 47ms/step - loss: 0.0594 - val_loss: 0.0424
Epoch 3/1000
25/25 [==============================] - 1s 48ms/step - loss: 0.0453 - val_loss: 0.0405
Epoch 4/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.0372 - val_loss: 0.0310
Epoch 5/1000
25/25 [==============================] - 1s 47ms/step - loss: 0.0317 - val_loss: 0.0298
Epoch 6/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.0279 - val_loss: 0.0280
Epoch 7/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.0261 - val_loss: 0.0316
Epoch 8/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.0250 - val_loss: 0.0274
Epoch 9/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.0239 - val_loss: 0.0281
Epoch 10/1000
25/25 [==============================] - 1s 47ms/step - loss: 0.0229 - val_lo

In [333]:
min(history.history["val_loss"])

0.016390856355428696

In [334]:
eval_dict = {"layer0": [100,100,50,50,50,50,"100(GRU)"],
            "layer1": [50,100,50,20,50,"50(GRU)", "50(GRU)"],
           "layer2": [10,50,10,10,20,20,20],
           "layer3": [0,10,0,0,0,0,0],
             "batch_size": [64,64,64,64,64,64,64],
           "mse": [0.0250, 0.0267,0.0196, 0.0264, 0.018311,0.0182, 0.0176]}

eval_df = pd.DataFrame.from_dict(eval_dict)
eval_df

,layer0,layer1,layer2,layer3,batch_size,mse
0,100,50,10,0,64,0.025000
1,100,100,50,10,64,0.026700
2,50,50,10,0,64,0.019600
3,50,20,10,0,64,0.026400
4,50,50,20,0,64,0.018311
5,50,50(GRU),20,0,64,0.018200
6,100(GRU),50(GRU),20,0,64,0.017600


In [379]:
def download_prediction_data(list_of_dfs):


    dt = datetime.now() - timedelta(hours= 49)

    dfs = []

    for df in list_of_dfs:

        sql = f"""
        SELECT *
        FROM `crypto-prediction-333213.crypto_BQDB.{df}`
        WHERE datetime > '{dt}';
        """

        dfs.append(pd.read_gbq(sql, project_id=project_id, dialect='standard'))

    return dfs

In [380]:
pred = download_prediction_data(list_of_dfs)

In [381]:
pred = data_cleaning(pred)

In [385]:
pred[3].head(10)

,price_doge,tweets_doge,hourly_pct_change,MA12_hours,MA72_hours,MAshort_over_long
datetime,,,,,,
2021-11-29 17:00:00+00:00,0.190363,2143,-99.000000,-99.0,-99.0,0
2021-11-29 18:00:00+00:00,0.191943,2178,0.008304,-99.0,-99.0,0
2021-11-29 19:00:00+00:00,0.192624,2310,0.003546,-99.0,-99.0,0
2021-11-29 20:00:00+00:00,0.190915,2056,-0.008870,-99.0,-99.0,0
2021-11-29 21:00:00+00:00,0.191732,1576,0.004277,-99.0,-99.0,0
2021-11-29 22:00:00+00:00,0.192286,1522,0.002892,-99.0,-99.0,0
2021-11-29 23:00:00+00:00,0.191243,1688,-0.005428,-99.0,-99.0,0
2021-11-30 00:00:00+00:00,0.190586,1595,-0.003432,-99.0,-99.0,0
2021-11-30 01:00:00+00:00,0.192032,2147,0.007587,-99.0,-99.0,0


In [353]:
len(pred)

19

In [354]:
pred[0].shape

(49, 6)

In [355]:
def reshape_data_for_prediction(data_to_be_predicted, list_of_scalers):
    # scaling the data & get_X_y for all datasets
    X = []

    for i in range(0,len(data_to_be_predicted)):

      df = data_to_be_predicted[i]
      scaler = list_of_scalers[i]
    
      # log transform
      price_col = [i for i in df.columns if "price" in i][0]
      df[price_col] = np.log(df[price_col])

      data = df.values
    
      X_arr = scaler.transform(data)

      X.append(X_arr)

    # stacking data three-dimensionally
    X = np.stack(X, axis= 2).reshape(len(X[0]),-1)

    X = np.expand_dims(X, axis= 0)

    return X

In [356]:
X = reshape_data_for_prediction(pred, scalers)

In [357]:
X.shape

(1, 49, 114)

In [358]:
predictions = X.copy()

In [367]:
pred_df = pd.DataFrame(model.predict(predictions).reshape(horizon, coins))

In [368]:
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.747655,0.516267,0.924419,0.607555,0.618774,0.885741,0.700631,0.791470,0.656149,0.825090,0.628357,0.547511,0.879545,0.824057,0.724533,0.759756,0.193392,0.863020,0.758486
1,0.748319,0.514939,0.927285,0.606969,0.619370,0.888979,0.702637,0.790771,0.652683,0.824252,0.632532,0.549220,0.880580,0.824065,0.729985,0.757565,0.190633,0.862424,0.758129
2,0.749659,0.510877,0.927964,0.607639,0.621351,0.886333,0.701562,0.791959,0.659725,0.821093,0.626748,0.552725,0.879573,0.823411,0.731038,0.760399,0.194430,0.862789,0.761029
3,0.750919,0.512285,0.924736,0.606522,0.615713,0.889715,0.706013,0.792060,0.661600,0.822710,0.625840,0.554412,0.878613,0.820587,0.729242,0.760110,0.197455,0.862705,0.758439
4,0.751738,0.511507,0.927185,0.606900,0.615203,0.888780,0.705543,0.789906,0.663632,0.821190,0.624167,0.555005,0.878332,0.822124,0.721496,0.755830,0.188990,0.862308,0.755177


# Inverse Transform

In [369]:
list_of_dfs = ["ban", "cummies", "dinu", "doge",
"doggy", "elon", "erc20", "ftm", "grlc", "hoge",
"lowb", "mona", "samo", "shib", "shibx", "smi",
"wow", "yooshi","yummy"]


def inverse_transformer(y, scaler):

  # inverse scaling
  y = np.c_[y, 1,1,1,1,1]
  y = scaler.inverse_transform(y)
  y= y[:,0]
  
  # inverse log-transforming
  y = np.exp(y)[0]
  return y

def reshape_predictions(prediction_dataframe, list_of_scalers, list_of_dfs):

    for i in range(0,prediction_dataframe.shape[1]):
    
      scaler = list_of_scalers[i]
      prediction_dataframe[i] = prediction_dataframe[i].apply(lambda x: inverse_transformer(x,scaler))
      
    # renaming columns
    prediction_dataframe.columns = list_of_dfs

    prediction_dict = prediction_dataframe.to_dict(orient="list")

    return prediction_dict

In [370]:
df_tmp = pred_df.copy()

In [371]:
scalers = joblib.load('scaler.joblib')

In [372]:
output_df = reshape_predictions(df_tmp, scalers, list_of_dfs)

In [386]:
pd.DataFrame.from_dict(output_df)

,ban,cummies,dinu,doge,doggy,elon,erc20,ftm,grlc,hoge,lowb,mona,samo,shib,shibx,smi,wow,yooshi,yummy
0,0.025293,0.039951,3.055666e-10,0.231615,0.006011,0.000001,0.031589,2.276873,0.054982,0.000441,0.000092,1.390302,0.119108,0.000047,0.001114,0.000004,0.206700,0.000002,0.00001
1,0.025324,0.039867,3.665782e-10,0.231544,0.006017,0.000001,0.031669,2.274862,0.054830,0.000441,0.000093,1.391301,0.119653,0.000047,0.001130,0.000004,0.206070,0.000002,0.00001
2,0.025387,0.039612,3.827316e-10,0.231625,0.006038,0.000001,0.031626,2.278281,0.055140,0.000437,0.000092,1.393351,0.119122,0.000047,0.001133,0.000004,0.206938,0.000002,0.00001
3,0.025446,0.039701,3.117710e-10,0.231489,0.005978,0.000001,0.031804,2.278571,0.055222,0.000439,0.000092,1.394339,0.118618,0.000046,0.001127,0.000004,0.207632,0.000002,0.00001
4,0.025484,0.039652,3.642456e-10,0.231535,0.005973,0.000001,0.031785,2.272379,0.055312,0.000437,0.000092,1.394687,0.118471,0.000047,0.001106,0.000004,0.205695,0.000002,0.00001
5,0.025506,0.039507,5.257774e-10,0.231260,0.005980,0.000001,0.031597,2.270349,0.055180,0.000436,0.000092,1.394754,0.118175,0.000047,0.001118,0.000004,0.207540,0.000002,0.00001
6,0.025457,0.039479,4.878278e-10,0.231532,0.005989,0.000001,0.031915,2.274334,0.055157,0.000435,0.000092,1.396935,0.118658,0.000047,0.001104,0.000004,0.207694,0.000002,0.00001
7,0.025504,0.039112,3.020398e-10,0.231153,0.006016,0.000001,0.031814,2.268549,0.055325,0.000434,0.000092,1.398163,0.118278,0.000047,0.001098,0.000004,0.207134,0.000002,0.00001
8,0.025565,0.039359,3.404492e-10,0.231279,0.006023,0.000001,0.031994,2.268156,0.055331,0.000434,0.000092,1.398921,0.118869,0.000047,0.001101,0.000004,0.206869,0.000002,0.00001
9,0.025383,0.039326,3.437183e-10,0.230874,0.006013,0.000001,0.031800,2.269105,0.055364,0.000433,0.000092,1.398748,0.118352,0.000047,0.001087,0.000004,0.207053,0.000002,0.00001


In [149]:
pred_df.to_dict(orient='list')

{'ban': [0.00828293087844601,
  0.008275028881449095,
  0.008274631546010646,
  0.008286461292471566,
  0.008261758385796696,
  0.008266238514758821,
  0.008272920742503439,
  0.008239031574660986,
  0.008242845729112562,
  0.008294953513494999,
  0.008297932989562424,
  0.00826674473758222,
  0.008261659606835224,
  0.008256786615762469,
  0.008282057387835532,
  0.008260173523408711,
  0.008283035831001874,
  0.008277029949788501,
  0.008249491587520525,
  0.008263838240967807,
  0.008299592619827009,
  0.00827519309777462,
  0.008254637974216114,
  0.00826917812975493],
 'cummies': [0.02298770695066484,
  0.022988880596593826,
  0.02308880333050652,
  0.023133346989105956,
  0.023182667297620196,
  0.02308887720791491,
  0.023254707297685126,
  0.023138064056220962,
  0.023279789790488593,
  0.023193727310268773,
  0.02321499253363217,
  0.023213459991213412,
  0.02328616928053179,
  0.02319398924003837,
  0.023286479370700656,
  0.023509089354487256,
  0.02325620809250929,
  0.0232

# Evaluate

In [ ]:
rmse = np.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))

print(f"RMSE = {rmse}")

plt.figure(figsize= (15,7))
plt.plot(real_stock_price, color = 'blue', label = "Real Price")
plt.plot(predicted_stock_price, color = 'red', label = "Predicted Price")
plt.title("Dogecoin Price Prediction")
plt.xlabel("Time")
plt.ylabel("Dogecoin Price")
plt.legend()

plt.show()

In [ ]:
def preprocess(df, scaler):
    """method that pre-process the data"""

    # log transforming the data
    df["high"] = np.log(df["high"])

    # selecting relevant column from df
    dataset = df.values

    # scaling the data
    dataset_scaled = scaler.fit_transform(dataset)

    dataset_scaled = dataset_scaled.reshape(1,dataset.shape[0],dataset.shape[1])

    return dataset_scaled, scaler


def inverse_scaler(pred, scaler):

  pred = inverse_transformer(pred)

  pred = np.exp(pred)

  return pred

In [ ]:
df_pred_preproc, scaler = preprocess(df_pred, scaler)

In [ ]:
pred = model.predict(df_pred_preproc)
prediction = inverse_scaler(pred, scaler)[0][0]

prediction

In [ ]:
df_pred_preproc.shape

In [ ]:
pred = model.predict(df_pred_preproc)[0][0]
pred

# Features

- historical price data
- google price data
- rendite (calculate daily return)
- kumulierte rendite (z.b. letzte 5 Tage)
- short term moving average (letzte 10 Tage)
- long term moving average (letzte 30 Tage)
- alternativ: binäres feature (liegt kurzfristiger MA über langfristigen MA)
- talib (financial indicators inkludieren)
- talib RSI (relative strength index)
- volumen inkludieren

- evtl. auf Klassifikationsproblem umsteigen